# Feature Backfill

### Import

In [1]:
import pandas as pd
import requests
import hopsworks
import os

/opt/anaconda3/envs/bds-mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#url = ('https://api.energidataservice.dk/dataset/Elspotprices?offset=0&start=2022-01-01T00:00&end=now&filter=%7B%22PriceArea%22:[%22DK1%22]%7D&sort=HourUTC%20DESC')

In [3]:
url = ('https://api.energidataservice.dk/dataset/Elspotprices?offset=0&start=2022-01-01T00:00&end=2023-12-31T23:59&filter=%7B%22PriceArea%22:[%22DK1%22]%7D&sort=HourUTC%20DESC')

In [4]:
data = requests.get(url).json()

In [5]:
data['records']

[{'HourUTC': '2023-12-31T22:00:00',
  'HourDK': '2023-12-31T23:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 200.309998,
  'SpotPriceEUR': 26.870001},
 {'HourUTC': '2023-12-31T21:00:00',
  'HourDK': '2023-12-31T22:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 213.729996,
  'SpotPriceEUR': 28.67},
 {'HourUTC': '2023-12-31T20:00:00',
  'HourDK': '2023-12-31T21:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 220.660004,
  'SpotPriceEUR': 29.6},
 {'HourUTC': '2023-12-31T19:00:00',
  'HourDK': '2023-12-31T20:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 260.100006,
  'SpotPriceEUR': 34.889999},
 {'HourUTC': '2023-12-31T18:00:00',
  'HourDK': '2023-12-31T19:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 295.51001,
  'SpotPriceEUR': 39.639999},
 {'HourUTC': '2023-12-31T17:00:00',
  'HourDK': '2023-12-31T18:00:00',
  'PriceArea': 'DK1',
  'SpotPriceDKK': 335.459991,
  'SpotPriceEUR': 45.0},
 {'HourUTC': '2023-12-31T16:00:00',
  'HourDK': '2023-12-31T17:00:00',
  'PriceArea': 'DK1',
  'S

In [6]:
electricity_df = pd.DataFrame(data['records'])

In [7]:
electricity_df

,HourUTC,HourDK,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2023-12-31T22:00:00,2023-12-31T23:00:00,DK1,200.309998,26.870001
1,2023-12-31T21:00:00,2023-12-31T22:00:00,DK1,213.729996,28.670000
2,2023-12-31T20:00:00,2023-12-31T21:00:00,DK1,220.660004,29.600000
3,2023-12-31T19:00:00,2023-12-31T20:00:00,DK1,260.100006,34.889999
4,2023-12-31T18:00:00,2023-12-31T19:00:00,DK1,295.510010,39.639999
...,...,...,...,...,...
17515,2022-01-01T03:00:00,2022-01-01T04:00:00,DK1,280.130005,37.669998
17516,2022-01-01T02:00:00,2022-01-01T03:00:00,DK1,338.059998,45.459999
17517,2022-01-01T01:00:00,2022-01-01T02:00:00,DK1,321.410004,43.220001
17518,2022-01-01T00:00:00,2022-01-01T01:00:00,DK1,307.350006,41.330002


In [8]:
electricity_df.drop('SpotPriceEUR', axis=1, inplace=True)

In [9]:
electricity_df['SpotPriceDKK_KWH'] = electricity_df['SpotPriceDKK'] / 1000

In [10]:
electricity_df.drop('HourUTC', axis=1, inplace=True)

In [11]:
electricity_df.drop('SpotPriceDKK', axis=1, inplace=True)

In [12]:
electricity_df.rename(columns={'HourDK': 'time'}, inplace=True)

In [13]:
electricity_df['time'] = electricity_df['time'].astype(str).str[:-3]

In [14]:
electricity_df

,time,PriceArea,SpotPriceDKK_KWH
0,2023-12-31T23:00,DK1,0.20031
1,2023-12-31T22:00,DK1,0.21373
2,2023-12-31T21:00,DK1,0.22066
3,2023-12-31T20:00,DK1,0.26010
4,2023-12-31T19:00,DK1,0.29551
...,...,...,...
17515,2022-01-01T04:00,DK1,0.28013
17516,2022-01-01T03:00,DK1,0.33806
17517,2022-01-01T02:00,DK1,0.32141
17518,2022-01-01T01:00,DK1,0.30735


### Note 
Hvis vi skal have tariffer med i modellen, anbefales det at vi sætter en faktor på 0.2 i tidsrummet 22 - 16 og en faktor på 0.6 eller 0.7 i tidsrummet 17 - 21.

In [15]:
url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")
# burde have enddate 2023-12-31. url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

response = requests.get(url)
print(response)

data = response.json()

weather_df = pd.DataFrame(data['hourly'])

<Response [200]>


In [16]:
weather_df

,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,2022-01-01T00:00,6.7,100,0.0,0.0,0.0,3,100,16.2,36.0
1,2022-01-01T01:00,6.6,100,0.0,0.0,0.0,3,100,16.2,30.2
2,2022-01-01T02:00,6.7,99,0.0,0.0,0.0,3,100,15.5,30.6
3,2022-01-01T03:00,6.7,100,0.0,0.0,0.0,3,100,12.7,28.8
4,2022-01-01T04:00,6.7,99,0.0,0.0,0.0,3,100,10.6,23.8
...,...,...,...,...,...,...,...,...,...,...
17515,2023-12-31T19:00,4.7,94,1.5,1.5,0.0,61,100,31.6,58.7
17516,2023-12-31T20:00,4.6,95,0.7,0.7,0.0,53,100,29.8,58.0
17517,2023-12-31T21:00,4.7,95,1.1,1.1,0.0,55,100,26.6,54.7
17518,2023-12-31T22:00,4.8,95,1.8,1.8,0.0,61,100,23.6,49.0


### Connecting to Hopsworks Feature Store

In [17]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


### Creating Feature Groups

In [18]:
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["time"],
    online_enabled=True,
)

In [19]:
weather_fg.insert(weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/706911


Uploading Dataframe: 100.00% |██████████| Rows 17520/17520 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x131d8ca10>, None)

In [20]:
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["time"],
    online_enabled=True,
)

In [21]:
electricity_fg.insert(electricity_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550040/fs/545863/fg/706912


Uploading Dataframe: 100.00% |██████████| Rows 17520/17520 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x131adaa90>, None)